In [168]:
import pandas as pd
import numpy as np
import folium

In [169]:
races_df = pd.read_csv('races.csv')
circuits_df = pd.read_csv('circuits.csv')

In [170]:
df = races_df.merge(circuits_df, on='circuitId', suffixes = ('_r','_c'))

In [171]:
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1079 entries, 0 to 1078
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   raceId       1079 non-null   int64  
 1   year         1079 non-null   int64  
 2   round        1079 non-null   int64  
 3   circuitId    1079 non-null   int64  
 4   name_r       1079 non-null   object 
 5   date         1079 non-null   object 
 6   time         1079 non-null   object 
 7   url_r        1079 non-null   object 
 8   fp1_date     1079 non-null   object 
 9   fp1_time     1079 non-null   object 
 10  fp2_date     1079 non-null   object 
 11  fp2_time     1079 non-null   object 
 12  fp3_date     1079 non-null   object 
 13  fp3_time     1079 non-null   object 
 14  quali_date   1079 non-null   object 
 15  quali_time   1079 non-null   object 
 16  sprint_date  1079 non-null   object 
 17  sprint_time  1079 non-null   object 
 18  circuitRef   1079 non-null   object 
 19  name_c

In [172]:
df.drop(['round', 'date', 'time',
       'url_r', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date',
       'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time',
       'url_c'], axis=1, inplace=True)

In [174]:
df['place'] = df.location + ', ' + df.country

In [175]:
df_old = df.loc[df.year<1960].groupby(['place','lat','lng'], as_index=False)['raceId'].count()
df_new = df.loc[df.year>2010].groupby(['place', 'country','lat','lng'], as_index=False)['raceId'].count()

In [176]:
m_old = folium.Map(location=[20,25], tiles="OpenStreetMap", zoom_start=2)
for i in range(0,len(df_old)):
    folium.Circle(
    location=[df_old.iloc[i]['lat'], df_old.iloc[i]['lng']],
    popup=df_old.iloc[i]['place'],
    radius=float(df_old.iloc[i]['raceId'])*30000,
    color='red',
    fill=True,
    fill_color='red'
    ).add_to(m_old)
    
m_old.save('map_old.html')
m_old

In [177]:
m_new = folium.Map(location=[20,25], tiles="OpenStreetMap", zoom_start=2)
for i in range(0,len(df_new)):
    folium.Circle(
    location=[df_new.iloc[i]['lat'], df_new.iloc[i]['lng']],
    popup=df_new.iloc[i]['place'],
    radius=float(df_new.iloc[i]['raceId'])*40000,
    color='red',
    fill=True,
    fill_color='red'
    ).add_to(m_new)
    
m_new.save('map_new.html')
m_new